# Foreacasting - Metric Per Entity

#### Version 1.0

### TODO

- Read aggreated data
- Forecast 30 years for metric per entity

### Observations

## Setup

### Import Packages

In [1]:
from datetime import datetime, date
from pprint import pprint
from collections import OrderedDict
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from prophet import Prophet

# Set ipython's max row display
# pd.set_option('display.max_row', 1000)
# Set iPython's max column width
pd.set_option('display.max_columns', 50)

# sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = [20, 10]

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

C:\Users\abdul\Anaconda3\envs\ds1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Functions

In [2]:
def masked_heatmap():
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    fig,ax = plt.subplots()
    sns.heatmap(corr,center = 0, fmt = ".3f", square = True, annot= True, mask=mask)

## Global Variables

#### Dataset Parameters

In [3]:
file_name = '../data/silver_tables/yearly_values_per_entity.csv'

year_range = [1960, 2022]
entity_category = 'Country'

forecast_horizon = 30

In [4]:
index_columns = [
    'Entity',
    'Code',
    'Year',
    'Entity_Category'
]
forecase_columns = [
    'Annual CO2 emissions',
    #'Annual change in primary energy consumption (%)',
    # 'Per capita electricity (kWh)',
    'Fossil fuels per capita (kWh)',
    # 'Fossil fuels (% equivalent primary energy)',
    # 'Renewables per capita (kWh - equivalent)', 'Renewables (% electricity)',
    # 'Solar Generation - TWh', 'prod of Electricity from solar (TWh)',
]

## Data Prep

In [5]:
df_original = pd.read_csv(file_name)

In [6]:
# df_original = df_original.loc[
#     (df_original['Year'] > year_range[0])
#     & (df_original['Year'] < year_range[1])
#     & (df_original['Entity_Category'] == entity_category)
# ][[*index_columns,*forecase_columns]]

df_original = df_original.loc[
    (df_original['Year'] > year_range[0])
    & (df_original['Year'] < year_range[1])
][[*index_columns,*forecase_columns]]

In [7]:
df_original.head(2)

,Entity,Code,Year,Entity_Category,Annual CO2 emissions,Fossil fuels per capita (kWh)
34,Aruba,ABW,1961,Country,645553.0,NaN
35,Aruba,ABW,1962,Country,708942.0,NaN


In [8]:
df_original['Year'] = pd.to_datetime(df_original['Year'], format='%Y')

### Create Per Country Dataframes

In [9]:
x = 'Year'
y = 'Annual CO2 emissions'

In [10]:
entity_dfs_dict = {}

for entity in df_original['Entity'].unique():
    entity_dfs_dict[entity] = df_original.loc[df_original['Entity'] == entity][[x, y]]
    entity_dfs_dict[entity].columns = ['ds', 'y']

In [11]:
entity_dfs_dict[entity].head()

,ds,y
21818,1961-01-01,5061263.0
21819,1962-01-01,4888959.0
21820,1963-01-01,4700377.0
21821,1964-01-01,4469069.0
21822,1965-01-01,5209198.0


## Forecast Generation 

In [12]:
def create_predict_df_for_entity(entity_name):
    df = entity_dfs_dict[entity_name]
    
    m = Prophet(
        yearly_seasonality='auto',
        weekly_seasonality=False,
        daily_seasonality=False,
    )
    m.fit(df)
    
    future = m.make_future_dataframe(periods=forecast_horizon, freq='Y')
    forecast = m.predict(future)
    
    df = df.set_index('ds').join(forecast.set_index('ds'), how='outer').reset_index()
    df['Entity'] = entity_name
    
    return df

In [13]:
def create_pred_df(entity_dfs_dict):
    all_pred_df = pd.DataFrame()

    try:
        for entity_name in entity_dfs_dict:
            print(f"Forecasting for {entity_name} done")
            
            try:
                df = create_predict_df_for_entity(entity_name)    
            except ValueError:
                print(f'Not enough observations in {entity_name} for forecasting')
                df = pd.DataFrame()

            all_pred_df = pd.concat([all_pred_df, df])
        return all_pred_df
    
    except KeyboardInterrupt:
        return all_pred_df

In [14]:
all_entities_pred_df = create_pred_df(entity_dfs_dict)

Forecasting for Aruba done
Forecasting for Afghanistan done
Forecasting for Angola done


INFO:prophet:n_changepoints greater than number of observations. Using 23.


Forecasting for Anguilla done
Forecasting for Albania done


INFO:prophet:n_changepoints greater than number of observations. Using 23.


Forecasting for Andorra done
Forecasting for Netherlands Antilles done
Not enough observations in Netherlands Antilles for forecasting
Forecasting for United Arab Emirates done
Forecasting for Argentina done
Forecasting for Armenia done


...

Forecasting for Timor done
Forecasting for Tonga done
Forecasting for Trinidad And Tobago done
Forecasting for Tunisia done
Forecasting for Turkey done


INFO:prophet:n_changepoints greater than number of observations. Using 23.


Forecasting for Tuvalu done
Forecasting for Taiwan done
Forecasting for Tanzania done
Forecasting for Uganda done
Forecasting for Ukraine done
Forecasting for Uruguay done
Forecasting for United States done
Forecasting for Uzbekistan done
Forecasting for Saint Vincent And The Grenadines done
Forecasting for Venezuela done
Forecasting for British Virgin Islands done
Forecasting for United States Virgin Islands done
Not enough observations in United States Virgin Islands for forecasting
Forecasting for Vietnam done
Forecasting for Vanuatu done


INFO:prophet:n_changepoints greater than number of observations. Using 15.


Forecasting for Wallis And Futuna done
Forecasting for Samoa done
Forecasting for Yemen done
Forecasting for South Africa done
Forecasting for Zambia done
Forecasting for Zimbabwe done


### Calculate Next 10 year sum

In [26]:
all_entities_pred_df['ds'].max()

Timestamp('2050-12-31 00:00:00')

In [27]:
df_final = all_entities_pred_df.rename(columns={'ds':'Year', 'y': f'{y}', 'yhat': f'{y}_predicted'})

In [28]:
df_final['Year'] = df_final['Year'].dt.year

In [29]:
df_final['Year'].max()

2050

In [30]:
entity_lookup = df_original[['Entity_Category', 'Entity']].drop_duplicates()

df_final = df_final.merge(entity_lookup, how='left')

In [31]:
df_final.head()

,Year,Annual CO2 emissions,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,Annual CO2 emissions_predicted,Entity,Entity_Category
0,1961,645553.0,348572.950385,-392061.276763,1.326828e+06,348572.950385,348572.950385,115998.488548,115998.488548,115998.488548,115998.488548,115998.488548,115998.488548,0.0,0.0,0.0,464571.438933,Aruba,Country
1,1962,708942.0,370307.263601,-374778.972297,1.302761e+06,370307.263601,370307.263601,99152.215880,99152.215880,99152.215880,99152.215880,99152.215880,99152.215880,0.0,0.0,0.0,469459.479482,Aruba,Country
2,1963,679088.0,392041.576818,-392316.399390,1.359247e+06,392041.576818,392041.576818,89219.703262,89219.703262,89219.703262,89219.703262,89219.703262,89219.703262,0.0,0.0,0.0,481261.280081,Aruba,Country
3,1964,660275.0,413775.890031,-382731.039051,1.356004e+06,413775.890031,413775.890031,86213.095529,86213.095529,86213.095529,86213.095529,86213.095529,86213.095529,0.0,0.0,0.0,499988.985560,Aruba,Country
4,1965,592387.0,435569.749307,-292298.458799,1.385045e+06,435569.749307,435569.749307,115998.488548,115998.488548,115998.488548,115998.488548,115998.488548,115998.488548,0.0,0.0,0.0,551568.237855,Aruba,Country


In [38]:
df_final.loc[df_final['Year'].isin([2010, 2011])]

,Year,Annual CO2 emissions,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,Annual CO2 emissions_predicted,Entity,Entity_Category
49,2010,2506176.0,1.414269e+06,5.772685e+05,2.346292e+06,1.414269e+06,1.414269e+06,9.915222e+04,9.915222e+04,9.915222e+04,9.915222e+04,9.915222e+04,9.915222e+04,0.0,0.0,0.0,1.513421e+06,Aruba,Country
50,2011,2498848.0,1.436003e+06,6.222454e+05,2.363576e+06,1.436003e+06,1.436003e+06,8.921970e+04,8.921970e+04,8.921970e+04,8.921970e+04,8.921970e+04,8.921970e+04,0.0,0.0,0.0,1.525223e+06,Aruba,Country
139,2010,8397779.0,4.482451e+06,2.664981e+06,8.573177e+06,4.482451e+06,4.482451e+06,1.025230e+06,1.025230e+06,1.025230e+06,1.025230e+06,1.025230e+06,1.025230e+06,0.0,0.0,0.0,5.507682e+06,Afghanistan,Country
140,2011,12105788.0,4.604341e+06,2.802753e+06,8.674368e+06,4.604341e+06,4.604341e+06,1.149124e+06,1.149124e+06,1.149124e+06,1.149124e+06,1.149124e+06,1.149124e+06,0.0,0.0,0.0,5.753465e+06,Afghanistan,Country
229,2010,28864503.0,2.669853e+07,1.584073e+07,2.788072e+07,2.669853e+07,2.669853e+07,-4.535899e+06,-4.535899e+06,-4.535899e+06,-4.535899e+06,-4.535899e+06,-4.535899e+06,0.0,0.0,0.0,2.216263e+07,Angola,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19236,2011,473948952.0,4.565131e+08,4.232050e+08,4.793291e+08,4.565131e+08,4.565131e+08,-3.157381e+06,-3.157381e+06,-3.157381e+06,-3.157381e+06,-3.157381e+06,-3.157381e+06,0.0,0.0,0.0,4.533557e+08,South Africa,Country
19326,2010,3120300.0,7.100872e+05,1.976791e+06,5.323809e+06,7.100872e+05,7.100872e+05,2.920004e+06,2.920004e+06,2.920004e+06,2.920004e+06,2.920004e+06,2.920004e+06,0.0,0.0,0.0,3.630091e+06,Zambia,Country
19327,2011,3372367.0,7.194968e+05,1.956958e+06,5.176817e+06,7.194968e+05,7.194968e+05,2.928404e+06,2.928404e+06,2.928404e+06,2.928404e+06,2.928404e+06,2.928404e+06,0.0,0.0,0.0,3.647901e+06,Zambia,Country
19416,2010,7878316.0,1.174671e+06,8.412342e+06,1.601006e+07,1.174671e+06,1.174671e+06,1.098231e+07,1.098231e+07,1.098231e+07,1.098231e+07,1.098231e+07,1.098231e+07,0.0,0.0,0.0,1.215698e+07,Zimbabwe,Country


In [40]:
[2021+x for x in range(10)]

[2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]

In [47]:
def calculate_x_years_forecast_sum(frame, n_years, start_year=2021):
    year_list = [start_year+x for x in range(n_years)]
    
    frame = frame.loc[frame['Year'].isin(year_list)]
    
    return frame.groupby('Entity')[f'{y}_predicted'].sum()

In [54]:
forecast_sum

Entity
Afghanistan          7.812264e+07
Albania              4.483901e+07
Algeria              1.826159e+09
Andorra              4.257528e+06
Angola               3.666501e+08
                         ...     
Wallis And Futuna    2.519330e+05
World                4.504659e+11
Yemen                2.212880e+08
Zambia               3.845097e+07
Zimbabwe             1.440840e+08
Name: Annual CO2 emissions_predicted, Length: 221, dtype: float64

In [53]:
forecast_sum = calculate_x_years_forecast_sum(
    df_final.set_index('Entity')[[f'{y}_predicted', 'Year']],
    10,
    2021
)

forecast_sum = forecast_sum.rename(f'{y}_predicted_10')

df_final.join(forecast_sum, on='Entity')

ValueError: columns overlap but no suffix specified: Index(['Annual CO2 emissions_predicted'], dtype='object')

In [51]:
df_final.head()

,Year,Annual CO2 emissions,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,Annual CO2 emissions_predicted,Entity,Entity_Category,Annual CO2 emissions_predicted_next_10_sum
0,1961,645553.0,348572.950385,-392061.276763,1.326828e+06,348572.950385,348572.950385,115998.488548,115998.488548,115998.488548,115998.488548,115998.488548,115998.488548,0.0,0.0,0.0,464571.438933,Aruba,Country,NaN
1,1962,708942.0,370307.263601,-374778.972297,1.302761e+06,370307.263601,370307.263601,99152.215880,99152.215880,99152.215880,99152.215880,99152.215880,99152.215880,0.0,0.0,0.0,469459.479482,Aruba,Country,NaN
2,1963,679088.0,392041.576818,-392316.399390,1.359247e+06,392041.576818,392041.576818,89219.703262,89219.703262,89219.703262,89219.703262,89219.703262,89219.703262,0.0,0.0,0.0,481261.280081,Aruba,Country,NaN
3,1964,660275.0,413775.890031,-382731.039051,1.356004e+06,413775.890031,413775.890031,86213.095529,86213.095529,86213.095529,86213.095529,86213.095529,86213.095529,0.0,0.0,0.0,499988.985560,Aruba,Country,NaN
4,1965,592387.0,435569.749307,-292298.458799,1.385045e+06,435569.749307,435569.749307,115998.488548,115998.488548,115998.488548,115998.488548,115998.488548,115998.488548,0.0,0.0,0.0,551568.237855,Aruba,Country,NaN


In [32]:
df_final['Entity'].value_counts()

Libya                91
Equatorial Guinea    91
Luxembourg           91
Latvia               91
Morocco              91
                     ..
Eritrea              57
Kosovo               51
Wallis And Futuna    50
Timor                49
Christmas Island     44
Name: Entity, Length: 222, dtype: int64

In [32]:
df_final['Entity'].value_counts()

Libya                91
Equatorial Guinea    91
Luxembourg           91
Latvia               91
Morocco              91
                     ..
Eritrea              57
Kosovo               51
Wallis And Futuna    50
Timor                49
Christmas Island     44
Name: Entity, Length: 222, dtype: int64

## Save Tables

In [25]:
df_final.to_csv('../data/forecasts_tables/all_entities_forecasts.csv', index=False)